In [1]:

import os
import sys
import re
from PIL import Image
import psycopg2
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import gc

# folderpath='C:/Users/heenatigalakanat3478/Saskatchewan Polytechnic/Peter Lucas Gravel Pit AI_GRP - General/ARD Grant/Data/Hillshade images/25x25/augmented'
# folderpath='C:/Users/heenatigalakanat3478/Saskatchewan Polytechnic/Peter Lucas Gravel Pit AI_GRP - General/ARD Grant/Data/Hillshade images/Augmented25x25_actualsize/'
folderpath='C:/Users/patel1637/MB_HS_100/'

'''use appropriate list_of_fs function according tot the folder structure'''


def list_of_fs(list_of_folders):
    list_of_files=[]
    for a in list_of_folders:
        temp_path=folderpath+'/'+a        
        list_of_files.append(temp_path)
    
    return list_of_files

def check_single_pit(a, pit_list):    
    #a is a list of coords
    
    for b in pit_list:
        # print('a:',a)
        # print('b:',b)
        # print(b[1])
        # print(a[0])
        # print(a[1])
        # if (abs(b[1]) >= abs(a[0])) :
        #     print("condition1 good")
        # if (abs(b[1]) <= abs(a[1])):
        #     print("condition2 good")
        # if (b[2] >= a[2]):
        #     print("condition3 good")
        # if (b[2] <= a[3]):
        #     print("condition4 good")
        if (b[1] >= a[0]) and (b[1] <= a[1]) and (b[2] >= a[2]) and (b[2] <= a[3]):
            print('Pit')
            
            
            return 1
    return 0    

In [2]:
def get_coords_image(img_path):
    #find the coordinates from the image name
    filename=img_path.split('/')[-1]
    filename=filename.split('-')
    print(filename)

    coords2 = [
        float(filename[2]) * -1,  # Convert to negative float
        float(filename[4]) * -1,  # Convert to negative float
        float(filename[5]),       # Ensure positive float
        float(filename[6]),       # Ensure positive float
        filename[7]
    ]
        
    return coords2,filename[0]


In [3]:
def save_image(tiles,file_names,img_path): 
    
    if len(tiles) != len(file_names):
        raise ValueError("Number of tiles and file names must be the same")
    c=0
    for idx, (tile, file_name) in enumerate(zip(tiles, file_names)):
        c+=1 
        file_path = os.path.join(os.path.dirname(img_path), f"{file_name}.jpg")
        # print('image split and saved:',c, file_path)
        # Save the tile
        tile.save(file_path)

In [4]:
def pit_list_db():
    dbname = "pointdb"
#     user = "deshan"
#     password = "peterpan" #super secure password
#     host = "208.79.218.126"
#     port = "5432"  # default PostgreSQL port is 5432
#     data_table="geo_points"
#     pit_data_table="pit_area_squares"
    pit_list=[]
    kms_needed=12.5
    km_per_degree=0.0090 #km per degree
    
    dbname = "pointdb"
    user = "jaydeep"
    password = "pointinfo" #super secure password
    host = "208.79.218.126"
    port = "5432"  # default PostgreSQL port is 5432
    data_table="geo_points"
    pit_data_table="pit_area_squares"

    query= f'select * from pit_list'

    try:
        conn = psycopg2.connect(
            dbname=dbname, 
            user=user, 
            password=password, 
            host=host, 
            port=port
        )

        # Create a cursor object https://www.psycopg.org/docs/usage.html
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch the results
        pit_list = cursor.fetchall()
        

        # Close the cursor and connection
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"An error occurred: {e}")
    
    return pit_list

In [5]:
def cut_img_names(list_coords,pit_nonpit_var):
    #list of coords is a,b,c,d a,b is xmin,xmax and c,d is ymin,ymax
    xmin=float(list_coords[0])
    xmax=float(list_coords[1])
    ymin=float(list_coords[2])
    ymax=float(list_coords[3])
    pit_nonpit_var=float(pit_nonpit_var)
    aug=list_coords[4]

    xmid=(xmin+xmax)/2
    ymid=(ymin+ymax)/2

    img1_coords=[xmin,xmid,ymin,ymid]
    img2_coords=[xmid,xmax,ymin,ymid]
    img3_coords=[xmin,xmid,ymid,ymax]
    img4_coords=[xmid,xmax,ymid,ymax]    
    img=[img1_coords,img2_coords,img3_coords,img4_coords]
    pit_nonpit_cut_imgs=[]
    pit_list=pit_list_db()
    for im in img:
        if pit_nonpit_var==1:
                print('Pit in 25x25')
                k=check_single_pit(im, pit_list)
                pit_nonpit_cut_imgs.append(k)
        else:
                pit_nonpit_cut_imgs.append(0)
        

    name1=str(pit_nonpit_cut_imgs[0])+'-'+'-'+str(xmin)+'-'+'-'+str(xmid)+'-'+str(ymin)+'-'+str(ymid)+'-'+aug
    name2=str(pit_nonpit_cut_imgs[1])+'-'+'-'+str(xmid)+'-'+'-'+str(xmax)+'-'+str(ymin)+'-'+str(ymid)+'-'+aug
    name3=str(pit_nonpit_cut_imgs[2])+'-'+'-'+str(xmin)+'-'+'-'+str(xmid)+'-'+str(ymid)+'-'+str(ymax)+'-'+aug
    name4=str(pit_nonpit_cut_imgs[3])+'-'+'-'+str(xmid)+'-'+'-'+str(xmax)+'-'+str(ymid)+'-'+str(ymax)+'-'+aug

    name_list=[name1,name2,name3,name4]



    return name_list


In [6]:
def split_image(img_path,save_path):
    im=Image.open(img_path)
    width, height = im.size
    M = width // 2  # Width of each tile
    N = height // 2  # Height of each tile
    # tiles = [im[x:x+M,y:y+N] for x in range(0,im.size[0],M) for y in range(0,im.size[1],N)]
    tiles = []  # List to store the cropped tiles
    for i in range(0, width, M):
        for j in range(0, height, N):
            # Define the bounding box for the tile
            left = i
            upper = j
            # if i==0 and j==0: 
            right = min(i + M, width)
            lower = min(j + N, height)
            # else:
            #     right=width
            #     lower=height
            # Crop the tile from the original image
            tile = im.crop((left, upper, right, lower))
            tiles.append(tile)
    cut_coords,pit_nonpit_var = get_coords_image(img_path)
    file_names =cut_img_names(cut_coords,pit_nonpit_var)  
    print("tiles",len(tiles)) 
    save_image(tiles, file_names,save_path)

In [7]:
def main():
    save_path='C:/Users/patel1637/MB_100_DIV/'
    list_of_folders = os.listdir(folderpath)
    list_of_files=list_of_fs(list_of_folders)
    # f_len=len(list_of_files)
    # f_len=f_len//2
    for a in list_of_files:        
        split_image(a,save_path)

    print('done')
    return 0

if __name__ == '__main__':
    main()

['1', '', '100.4396111', '49.89561824', '383.0', '100.jpg']


ValueError: could not convert string to float: '100.jpg'